##  <center><span style = 'color:#0b5345'> Eway Bill Data Analysis for Road-to-Rail Transportation Conversion & Infrastructure Development Prioritization </span>

### <span style = 'color:#0b5345'> Abstract: </span>
 
The aim of this project is to conduct a comprehensive analysis of the eway bill data to gain valuable insights into the transportation activities taking place through roadways. The project focuses on two key objectives: identifying potential companies for persuading transportation conversion from road to rail and prioritizing infrastructure development initiatives.

### <span style = 'color:#0b5345'> Introduction: </span>

In today's fast-paced world, the efficient movement of goods and people is of paramount importance for economic growth and sustainability. As concerns surrounding environmental impact and road congestion intensify, railways present a compelling alternative as a more sustainable and cost-effective means of transportation. This analysis focuses on the imperative of improving railways and attracting major companies to increase the rail coefficient, thereby promoting the broader adoption of rail transportation for goods movement.

### <span style = 'color:#0b5345'> Objectives: </span>

### <span style = 'color:#0b5345'> Road Rail conversion: </span>
Through exploration of the data, companies that predominantly use road transportation for goods movement can be identified by quantifying their transportation and understanding their transportation patterns. These companies can be approached to know the reason for not choosing Railways as their mode of logistics. By pitching in the advantages of transporting their consignments through Rail and by addressing their concerns while choosing Railways as a transportation partner, the company can be transitioned to use Railways for their consignment transportation.

### <span style = 'color:#0b5345'> Infrastructure Development Prioritization: </span>
Using clustering techniques and analysis, major transport originations, destinations, major commodities being transported etc, can be identified to prioritize infrastructure developments. The=is identification of critical transportation nodes will enable Railways to allocate resources efficiently, directing investments towards projects that offer the most significant impact on the movement of goods.

### <span style = 'color:#0b5345'> Description of Data: </span>

EWay Bill is an Electronic Way bill for movement of goods to be generated on the eWay Bill Portal. A GST registered person cannot transport goods in a vehicle whose value exceeds Rs. 50,000 (Single Invoice/bill/delivery challan) without an e-way bill. The dataset contains the following columns

<b>From GSTIN & Name</b> - GST number and name of the company transporting/owning the goods<br>
<b>To GSTIN & Name</b> - GST number and name of the company to which the goods are being transported<br>
<b>From Place & Pin</b> - Address of the consignment transporting company<br>
<b>To Place & Pin</b> - Address of the consignment receiver<br>
<b>EWB No. & Dt.</b> - Eway bill number and date of transportation<br>
<b>Doc No. & Dt.</b> - Document number and date of generation<br>
<b>Assess Val.</b> - Value of consignment<br>
<b>Tax Val.</b> - GST of consignment<br>
<b>HSN Code.</b> - HSN code of consignment<br>
<b>HSN Desc.</b> - Detailed description of the consignment<br>
<b>veh_no</b> - The vehicle number in which the consignment will be transported

A dummy dataset has been used to showcase the skills and to maintain data integrity. The dummy dataset used for the analysis is created for a 4 day period having following dimensions.

In [5]:
#to get the number of rows and columns of a dataset
eway.shape

(31814, 12)

## <span style = 'color:#0b5345'> DATA CLEANING </span>

In [3]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#loading the dataset
eway = pd.read_excel('eway base data.xlsx')

In [3]:
pd.set_option('display.max_rows',100)
pd.set_option('display.max_seq_items',6000)

In [5]:
# remove unwanted columns - s.no, tax value, hsn description
eway.drop(['S.No.','Tax Val.','HSN Desc.'],axis=1,inplace = True)

In [6]:
# splitting gst number form the gst number and name column
eway['From_gst']=eway['From GSTIN & Name'].str.split('/',expand=True)[0]

In [7]:
#removing records of GST unregistered companies as these will not have the pontential
#to earn revenue to railways
eway.drop(eway[eway.From_gst.str.len()==4].index,inplace = True)

In [8]:
#removing rows with null values in vehicle numbers as it is mandatory for a vehicle moving in road to have vehicle number
#rows with empty values in vehicle numbers pertain to other modes of transport

#getting the index of the rows with null vehicle numbers
nullidx = eway[eway.veh_no.isna()].index

# to remove the null rows
eway.drop(nullidx,inplace= True)

#resetting the index
eway.reset_index(drop=True,inplace=True)

### <span style = 'color:#0b5345'> CLEANING COMPANY NAMES </span>

In [9]:
clean1=eway['From GSTIN & Name'].str.split('/',n=1,expand=True)[1]

clean2 = clean1.str.rsplit('/',n=1,expand=True)[0].str.strip()

clean2 = clean2.str.upper()

#making the word pvt common as private in all the company names
clean2 = clean2.str.replace('PVT.','PRIVATE')
clean2 = clean2.str.replace('PVT','PRIVATE')
clean2 = clean2.str.replace('PRIVATE ','PRIVATE')
clean2 = clean2.str.replace('PRIVATE','PRIVATE ')

#making the word ltd common as limited in all the company names

clean2 = clean2.str.replace('LTD.','LIMITED')
clean2 = clean2.str.replace('LTD','LIMITED')
clean2 = clean2.str.replace('LIMITED ','LIMITED')


clean3=clean2.str.replace('  ',' ')

#checking the company names after a comma split
#most of the company names were present before the , only a few company names has been mentioned after comma
t1 = clean3.str.split(',',n=1,expand=True)[1]

#list of company names after comma - first filtered the t1 to contain only company names and not anyother, from checking the filtered
#items manually, final filtered list has been obtained
tochange =['SHUNMUGA TEX', 'ROBERT FIBRE INDUSTRIES','PROP SREE MURUGAN PLASTICS', 'MUTHUSAMY TEXTILES','TVL.SRI BALAJI INDUSTRIES', 'BABU AGENCIES',  'LAKSHMI PIPE']

#to replace the company names column with right company names
for idx,i in enumerate(clean3):
    for j in tochange:
        if j in i:
            clean3.iloc[idx] = j
        
clean3 = clean3.str.split(',',n=1,expand=True)[0]

clean3 = clean3.str.replace(' P LIMITED',' PRIVATE LIMITED')

clean3 = clean3.str.replace('[MADHAVARAM]','',regex = False)

eway['From_company'] = clean3

C:\Users\Sriram\AppData\Local\Temp\ipykernel_11800\1925582732.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  clean2 = clean2.str.replace('PVT.','PRIVATE')
C:\Users\Sriram\AppData\Local\Temp\ipykernel_11800\1925582732.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  clean2 = clean2.str.replace('LTD.','LIMITED')


In [10]:
eway["From_gst"] = eway.From_gst.str.strip()

#checking how many unique duplicated values are there
eway.From_company.value_counts().count() - eway.From_gst.value_counts().count()

183

In [11]:
#grouping gst with company names to find the company names that are repeated for a single GST

#creating a dummy column to make the grouping process easier
eway["dummy"]=1
groupedgst = pd.DataFrame(eway.groupby(['From_gst','From_company'])['dummy'].count())
groupedgst = pd.DataFrame(groupedgst.dummy.index.to_list())

In [12]:
#taking the GST Nos. where company names are different for the same GST
company_duplicates_gst = groupedgst[0].value_counts()[groupedgst[0].value_counts()>1].index

In [13]:
#taking the list of GST nos. and company names that are duplicated
dup_list = eway[eway.From_gst.isin(company_duplicates_gst)][['From_gst','From_company']].drop_duplicates().sort_values('From_gst')

In [14]:
#creating a dictionary of gst numbers with list of company names
#to manually check which is the most appropriate company name that can be assigned to the gst number
dup_gst = dup_list.From_gst.unique()
gst_with_dup_comp = {}

for ind,i in enumerate(dup_gst):
    comp = dup_list[dup_list.From_gst == i]['From_company'].values
    print(ind,i,':',comp)
    gst_with_dup_comp.update({i:list(comp)})


#manually numbering the index (with the right company name) of the list present in the value part of the dictionary
#in the same order as the key-value pair
sel_names= [4,1,0,1,0,2,2,0,1,0,3,0,0,1,0,2,1,1,0,0,0,0,1,0,1,0,4,1,0,1,1,0,0,3,3,1,0,0,1,1,0,0,1,1,2,0,0,0,1,1,1,1,0,0,0,0,1,0,3,2,1,0,2,2,0,0,1,3,2,1,1,1,1,0,2,1,1,0,0,2,1,0,0,0,0,1,1,0,0,1,0,2,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,2,1,0,1,0,1,0,3,2,0,0,0,1,0,1,0,1,2,0,1,1,1,1,1,1,1,3,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,2,0,0,1,1,0,1,1]

#Gst nos. which did not have a proper company name
#33AAACH3005M1ZY, 33AAACJ5438J1ZL, 33AAACM4509P1ZC, 33AAACT7987L1ZP, 33AAACW1645G1ZM, 33AABCC2461K1ZW, 33AABCD8855E1ZQ, 33AACCA2744G1Z4, 33AADCB1921F1ZB, 33AADCS5971L1ZZ, 33AAFCN9740E1ZI, 33AAIFA8010E1Z1,33AVIPB6061B3Z5, 33DHVPP5660Q1Z8


0 33AAACA3622K1Z2 : ['APL CHENNAI CHROMPET APPS NO 57 ADAM NAGAR ROAD'
 'APL CHENNAI VANUR UPCOUNTRY OLD SV NO 138/4 NEW NO 74/2'
 'APL METRO DC CHENNAI NO 9' 'APL MADURAI SURVEY NO 273/2A'
 'ASIAN PAINTS LIMITED' 'APL-DBU-SRIPERUMBUDUR']
1 33AAACA8832H1ZV : ['7604 PPG ASIAN PAINTS PRIVATE LIMITEDPPG ASIAN PAINTS PRIVATE LIMITED'
 'PPG ASIAN PAINTS PRIVATE LIMITED']
2 33AAACA9647E1ZU : ['L&T VALVES LIMITED' 'LNT VALVES LIMITED']
3 33AAACB2533Q1ZP : ['MIDRANGE COMPONENTS (UNIT 2)' 'BRAKES INDIA PRIVATE LIMITED']
4 33AAACB3145L1ZX : ['BLOW PACKAGING INDIA PRIVATE LIMITED'
 'BLOW PACKAGING (INDIA) PRIVATE LIMITED']
5 33AAACB3202A1ZU : ['BIO RAD LABORATORIES(INDIA) PRIVATE LIMITED' ''
 'BIO-RAD LABORATORIES INDIA PRIVATE LIMITED']
6 33AAACB3754B1ZB : ['CAVINKARE PRIVATE LIMITEDCHE DEPOT' ''
 'CAVINKARE PRIVATE LIMITEDERD MFG']
7 33AAACB8772D1ZV : ['BULL MACHINES PRIVATE LIMITED' 'BULL PLANT I']
8 33AAACC3000F1ZN : ['CHEMPLAST SANMAR LIMITED- PLANT 3' 'CHEMPLAST SANMAR LIMITED- PLANT 2']
9 

In [15]:
#assigning each key in dictionary to only one value from the list of different company names
for ind,i in enumerate(gst_with_dup_comp.items()):
    print(ind, i[0], i[1][sel_names[ind]])
    gst_with_dup_comp[i[0]] = i[1][sel_names[ind]]
    

0 33AAACA3622K1Z2 ASIAN PAINTS LIMITED
1 33AAACA8832H1ZV PPG ASIAN PAINTS PRIVATE LIMITED
2 33AAACA9647E1ZU L&T VALVES LIMITED
3 33AAACB2533Q1ZP BRAKES INDIA PRIVATE LIMITED
4 33AAACB3145L1ZX BLOW PACKAGING INDIA PRIVATE LIMITED
5 33AAACB3202A1ZU BIO-RAD LABORATORIES INDIA PRIVATE LIMITED
6 33AAACB3754B1ZB CAVINKARE PRIVATE LIMITEDERD MFG
7 33AAACB8772D1ZV BULL MACHINES PRIVATE LIMITED
8 33AAACC3000F1ZN CHEMPLAST SANMAR LIMITED- PLANT 2
9 33AAACC9186C1ZU MERINO INDUSTRIES LIMITED.
10 33AAACC9497N1Z1 C R I PUMPS PRIVATE LIMITED
11 33AAACE3882D1ZZ EICHER MOTORS LIMITED
12 33AAACE5767F1ZR EUREKA FORBES LIMITED
13 33AAACE7066P1ZA ENDURANCE TECHNOLOGIES LIMITED
14 33AAACF0483F1Z3 EMERSON PROCESS MANAGEMENT CHENNAI PRIVATE LIMITED
15 33AAACF1034E1ZG M/S. FIEM INDUSTRIES LIMITED
16 33AAACF3357Q1ZD SCHAEFFLER INDIA LIMITED
17 33AAACF5248E1Z0 FLEXTRONICS TECHNOLOGIES (INDIA) PRIVATE LIMITED
18 33AAACG1395D1Z1 GODREJ & BOYCE MFG. CO. LIMITED
19 33AAACG1994N1ZB GABRIEL INDIA LIMITED-HOSUR
20 33AA

In [16]:
#replacing the companies names of GST nos. with the selected company names in the original dataset
for i,j in gst_with_dup_comp.items():
    for x in range(0,len(eway)):
        if eway.From_gst[x]== i:
            eway.iloc[x,10]= j

In [ ]:
#There are company names with no values present in it. It was observed that the company name was not available in the base data itself.

In [17]:
#splitting the 'To GSTIN & Name' to get To_GST column separately
eway['To_GST']=eway['To GSTIN & Name'].str.split('/',expand=True)[0]

### <span style = 'color:#0b5345'> WEBSCRAPPING COMPANY NAMES USING GST NUMBER </span>

An alternate approach available instead of cleaning all the company names (might still have small chances of having a wrong value despite cleaning intensively) is to use the gst number (unique values) and webscrape the company names from websites. Here, webscrapping has been made from 3 websites, prominently from the first, followed by another sites to fill in the null values obtained while scraping.

In [29]:
#webscraping using selenium to get the company names with GST number 
#so that the company name remains unique avoiding text errors in the given company name in the dataset
unique_gst = eway.To_GST.str.strip().unique().tolist()
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get('https://findgst.in/')
To_comp_names = dict()
for i in range(0,len(unique_gst)):
    To_comp_names.update({unique_gst[i]:searching(unique_gst[i])})
driver.close()

In [65]:
#GSTs with null values
remaining_gsts = pd.Series(To_comp_names)[pd.Series(To_comp_names).isna()].index

In [71]:
driver=webdriver.Chrome()
driver.get('https://cleartax.in/gst-number-search/')
driver.implicitly_wait(10)

for i in range(0,len(remaining_gsts)):
    To_comp_names.update({remaining_gsts[i]:searching_remaining(remaining_gsts[i])})
driver.close()

In [108]:
#GSTs with empty ('') values
empty_gst = pd.Series(To_comp_names)[pd.Series(To_comp_names)==''].index

In [111]:
#scrapping from the original GST site entering the captcha's manually
driver=webdriver.Chrome()
for i in range(0,len(empty_gst)):
    To_comp_names.update({empty_gst[i]:searching_with_captcha(empty_gst[i])})
driver.close()

In [24]:
#the government website to take company names using GST numbers is protected with captcha - using other easily available options
#scraping the GST values majorly from findgst.in
def searching(gst):     
    
    search = driver.find_element('id','gstnumber')
    search_button = driver.find_element(By.CSS_SELECTOR, 'input[value="Search GST number"]')
    search.send_keys(gst)
    search_button.click()
    try:
        #company=(driver.find_element(By.CSS_SELECTOR, 'small[class = "text-s-20 text-font-500 font-medium sm:text-base"]')
         #   .get_attribute('innerHTML'))
            
        company = driver.find_elements(By.TAG_NAME,'td')[3].get_attribute('innerText')
    except:
        company=np.NAN
    
    return company

In [70]:
#scraping values left out by findgst.in in cleartax.in 
def searching_remaining(gst):     
    
    search = driver.find_element('id','input')
    search_button = driver.find_element(By.CSS_SELECTOR, 'button[class="w-2/12 text-base py-4 px-8 text-center justify-center inline font-semibold btn-blue sm:w-full sm:mt-2 rounded-none rounded-tr-md rounded-br-md sm:rounded md:px-4"]')
    search.send_keys(gst)
    search_button.click()
    try:
        company=(driver.find_element(By.CSS_SELECTOR, 'small[class = "text-s-20 text-font-500 font-medium sm:text-base"]')
          .get_attribute('innerHTML'))
            
    except:
        company=np.NAN
    search.clear()
    
    return company

In [110]:
#for null gst (small portion of left out gst) from original government site entering captcha manually
def searching_with_captcha(gst):     
    driver.get('https://services.gst.gov.in/services/searchtp')
    driver.implicitly_wait(10)
    search = driver.find_element('id','for_gstin')
    search.send_keys(gst)
    time.sleep(6) #to manually enter captcha
    search_button = driver.find_element(By.CSS_SELECTOR, 'button[class="btn btn-primary "]')
    search_button.click()
    time.sleep(2) #waiting for the results to load
                
    try:
        company=driver.find_elements(By.CSS_SELECTOR, 'div[class = "col-sm-4 col-xs-12"]')[1].find_elements(By.TAG_NAME,'p')[1].get_attribute('innerText')
            
    except:
        company=np.NAN
    
    return company

In [118]:
#converting the obtained GSTs to a csv for future use
pd.Series(To_comp_names).to_csv('GSTnumbers_n_comp')

In [18]:
To_company = pd.read_csv('GSTnumbers_n_comp',index_col=0).to_dict()['0']

In [19]:
#mapping the dictionary of unique gst values and companies to a new column for all the gst values in the dataset
eway['To_GST']=eway.To_GST.str.strip()
eway['To_company']=eway.To_GST.map(To_company)

In [21]:
#remove unwanted columns (from gst and name, to gst and name, dummy varible)
eway.drop(columns=['From GSTIN & Name','To GSTIN & Name','dummy'],inplace=True)

### <span style = 'color:#0b5345'> CLEANING AND ADDING REQUIRED COLUMNS FROM OTHER SOURCES </span>

The HSN description column did not specify in general about the commodity rather it was a long description. Therefore to identify the type of commodity transported the first 2 characters of the HSN code which represents the chapter have been taken and its respective chapter name has been added to the dataset from an external source.

Similarly, the pin number has been taken from the address to identify the district of the same. The district names have been merged to this dataset obtained from an external source (India post website)

In [22]:
#split HSN code and map it with chapter name
#extracting first 2 values from hsn code
eway['HSN_chapter'] = eway['HSN Code'].map(lambda x: str(x)[0:2])

hsnchaps = pd.read_excel('hsn_chapters.xlsx',index_col=0).to_dict()['description']
eway['HSN_chapter']=eway.HSN_chapter.astype('int')
eway['HSN_Chapter_Name'] = eway.HSN_chapter.map(hsnchaps)

In [23]:
#splitting the pin from 'From Place & Pin'
eway['From_Pin']=eway['From Place & Pin'].str.rsplit('/',n=1,expand=True)[1].str.strip()

In [24]:
#splitting the pin from 'To Place & Pin'
eway['To_Pin']=eway['To Place & Pin'].str.rsplit('/',n=1,expand=True)[1].str.strip()

In [69]:
#reading the file with pincodes and removing the duplicates
pincodes =pd.read_csv('pincode.csv')
pincodes = pincodes[~pincodes.duplicated(subset=['Pincode'])]

C:\Users\Sriram\AppData\Local\Temp\ipykernel_11800\2489152267.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pincodes =pd.read_csv('pincode.csv')


In [70]:
#splitting the date from EWB No. & Dt. column  and converting the datatype

eway['date']=eway['EWB No. & Dt.'].str.rsplit('-',n=1,expand=True)[1].str.strip()
eway['date']=eway['date'].str.rsplit(' ',n=1,expand=True)[0].str.strip()
eway['date']= pd.to_datetime(eway['date'],format='%d/%m/%Y',errors='ignore')

In [71]:
eway['From_Pin']=eway.From_Pin.astype('int')

In [73]:
eway=eway.merge(pincodes,left_on='From_Pin',right_on = 'Pincode',how='left')

In [75]:
#renaming columns to mark it as orgination
eway.rename(columns={'District':'From_District','StateName':'From_State','Latitude':'From_Latitude','Longitude':'From_Longitude'},inplace=True)

In [76]:
eway['To_Pin']=eway.To_Pin.astype('int')
eway = eway.merge(pincodes,left_on='To_Pin',right_on='Pincode',how='left')

In [78]:
#renaming columns to mark it as destination
eway.rename(columns={'District':'To_District','StateName':'To_State','Latitude':'To_Latitude','Longitude':'To_Longitude'},inplace=True)

In [79]:
eway.drop(columns=['Pincode_y','Pincode_x','From Place & Pin','To Place & Pin','EWB No. & Dt.','Doc No. & Dt.'],inplace=True)

<b>Rearranging the column names</b>

In [85]:
eway = eway[['date','From_gst','From_company','From_Pin', 'From_District', 'From_State', 'From_Latitude',
       'From_Longitude','To_GST', 'To_company', 'To_Pin','To_District', 'To_State', 'To_Latitude',
       'To_Longitude','HSN Code','HSN_chapter', 'HSN_Chapter_Name','Assess Val.', 'veh_no' ]]

In [86]:
eway.to_csv('eway_80_cleaned.csv',index=False)

In [28]:
eway = pd.read_csv('eway_80_cleaned.csv')

In [29]:
eway

,date,From_gst,From_company,From_Pin,From_District,From_State,From_Latitude,From_Longitude,To_GST,To_company,To_Pin,To_District,To_State,To_Latitude,To_Longitude,HSN Code,HSN_chapter,HSN_Chapter_Name,Assess Val.,veh_no
0,2021-08-01,33AABCH1950J1ZT,HENSEL ELECTRIC INDIA PRIVATE LIMITED,631604,KANCHIPURAM,TAMIL NADU,12.852200,79.874700,37AAJCG0251F1ZV,GMD GREEN POWER PRIVATE LIMITED,502296,SANGAREDDY,TELANGANA,NaN,NaN,85371000,85,Electrical Parts & Electronics,402085.00,AP10W6935
1,2021-08-03,33BZOPS5991N1Z9,SABARI TEXTILE MILLS,625009,MADURAI,TAMIL NADU,12.145000,78.046500,37ADJPC8924D1ZM,BALAJI TENT WORKS,522003,GUNTUR,ANDHRA PRADESH,16.475417,80.741111,5209,52,"Cotton materials, Synthetics & Woven fabrics",45430.00,TN551422
2,2021-08-01,33AAACS7032B1ZZ,T V S MOTOR COMPANY LIMITED,635109,KRISHNAGIRI,TAMIL NADU,12.747362,77.842436,37AEHFS8703B1Z2,SLN AUTOMOBILES LLP,516434,Y.S.R.,ANDHRA PRADESH,15.056523,78.302443,85113020,85,Electrical Parts & Electronics,656.90,KA51AA8494
3,2021-08-01,33ABJFS8110P1ZS,STAR PLASTICS,638102,ERODE,TAMIL NADU,11.342255,77.727456,37BSXPA5892E1Z7,HI TECH FURNITURES,515411,ANANTAPUR,ANDHRA PRADESH,14.910000,78.000000,39259090,39,"Polymers, Polyethylene, Cellulose",118219.00,TN52F5143
4,2021-08-01,33AABCM8375L2Z2,THE RAMCO CEMENTS LIMITED,621713,PERAMBALUR,TAMIL NADU,11.150000,79.060000,37BSUPK3391P1ZN,KBC ENGINEERING WORKS,524001,SPSR NELLORE,ANDHRA PRADESH,14.451806,79.980917,25232910,25,Salts & Sands,75000.00,TN61Q2146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30674,2021-08-02,33AABCB7140C1ZC,BMW INDIA PRIVATE LIMITED,603002,CHENGALPATTU,TAMIL NADU,12.701279,80.010161,06AAGCD4246E1ZX,DEUTSCHE CARS PRIVATE LIMITED,121003,FARIDABAD,HARYANA,77.264444,28.462500,870323,87,Tractors & Motor Vehicles,3356582.39,NL01K8004
30675,2021-08-02,33AACCJ0197Q1Z7,JM FRICTECH INDIA PRIVATE LIMITED,602105,THIRUVALLUR,TAMIL NADU,12.967468,79.941817,06AAACE0074B1ZE,ESCORTS LTD,121007,FARIDABAD,HARYANA,28.383278,77.325889,8708,87,Tractors & Motor Vehicles,138351.60,HR38R9694
30676,2021-08-04,33AAAFT3874B1ZI,THE BUSINESS CENTRE,625107,MADURAI,TAMIL NADU,9.927083,78.239037,06AAACF9636N1Z6,FRESENIUS MEDICAL CARE INDIA PVT LTD,124105,JHAJJAR,HARYANA,28.597164,76.783619,90189031,90,"Medical, Chemical & Astronomy",571638.00,HR55AH2576
30677,2021-08-04,33AAACB2533Q1ZP,BRAKES INDIA PRIVATE LIMITED,600050,CHENNAI,TAMIL NADU,13.098861,80.183833,06AAACE0074B1ZE,ESCORTS LTD,121002,FARIDABAD,HARYANA,28.405301,77.374447,87083000,87,Tractors & Motor Vehicles,157675.80,HR38T8394


In [30]:
eway['date']= pd.to_datetime(eway['date'],errors='ignore')

In [40]:
#replacing wrong entries in company names for URP to null
eway.To_company = np.where(eway.To_GST == 'URP',np.NAN,eway.To_company)
eway.From_company = np.where(eway.From_gst == 'URP',np.NAN,eway.From_company)

In [37]:
#removing rows with value of transported goods less than 50000 as moving lower quantities of items in train are not feasible

#getting the indices
lowidx = eway[(eway['Assess Val.']<50000)].index

#removing the rows
eway.drop(lowidx,inplace = True)

In [49]:
Fromgst_nulls = {'33AABCR2655Q1Z1':'RECKITT BENCKISER INDIA PRIVATE LIMITED','33AACCI1509D1Z9':'INDIAN TERRAIN FASHIONS LTD','33AAGFL0864P1ZZ':'LAKSHMI NARAYANA PACKS LLP','33ACWFS0645R1Z9':'SUNSHINE PYRO PARK','33CNJPD9961P1ZA':'RUBHA OVERSEAS','33AAGCG1073L2ZM':'GOEL SPECIAL STEELS & ENGINEERING PRIVATE LIMITED'}

In [53]:
#replacing null values with company names
for i,j in Fromgst_nulls.items():
    eway.From_company = np.where(eway.From_gst == i,j,eway.From_company)

In [58]:
#all the nulls is To_company are Unregistered persons
eway[eway.To_company.isna()].To_GST.unique()

array(['URP'], dtype=object)

In [62]:
eway[eway.HSN_Chapter_Name.isna()]

,date,From_gst,From_company,From_Pin,From_District,From_State,From_Latitude,From_Longitude,To_GST,To_company,To_Pin,To_District,To_State,To_Latitude,To_Longitude,HSN Code,HSN_chapter,HSN_Chapter_Name,Assess Val.,veh_no
1837,2021-08-03,33CIWPP5468H2ZH,P4J CREATIONS,600007,CHENNAI,TAMIL NADU,13.150194,80.438083,37AAACH0945G1ZS,M/S HATSUN AGRO PRODUCT LTD,517124,CHITTOOR,ANDHRA PRADESH,13.383900,79.082100,998599,99,NaN,61026.0,TN01BJ7472
6341,2021-08-03,33AABCH5585P1Z2,KADIMI SPECIAL STEELS PRIVATE LIMITED,602105,THIRUVALLUR,TAMIL NADU,12.967468,79.941817,37AADCM5441K1ZA,MANGAL INDUSTRIES LIMITED,517124,CHITTOOR,ANDHRA PRADESH,13.383900,79.082100,998898,99,NaN,315052.0,TN23BH0279
10186,2021-08-03,33AHPPG7024G1ZH,SRI KRISHNA TRADING,638011,ERODE,TAMIL NADU,11.337861,77.707150,37DJRPA7941D1Z1,GODLINE PROCESS,517590,CHITTOOR,ANDHRA PRADESH,13.321194,79.578598,998821,99,NaN,836645.0,TN86E9281
16169,2021-08-04,33AABCR0347P1ZA,REDINGTON INDIA LIMITED,601201,THIRUVALLUR,TAMIL NADU,13.438000,80.169000,07AAKCA0785H1ZK,ALGORT TECHNOSOLUTIONS PRIVATE LIMITED,110014,SOUTH,DELHI,28.580000,77.260000,998713,99,NaN,200000.0,DL01LR2397


In [76]:
missing_dist_from = eway[eway.From_District.isna()].From_Pin.unique()

In [37]:
missing_dist_to =eway[eway.To_District.isna()].To_Pin.unique()

In [39]:
leftout_pins = pd.read_excel('pincode leftout.xlsx')

C:\Users\Sriram\AppData\Local\Temp\ipykernel_632\170784215.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  leftout_pins_from =leftout_pins_from[~(leftout_pins[leftout_pins.Pincode.isin(missing_dist_from)].duplicated(subset=['Pincode', 'District', 'State']))]


In [ ]:
leftout_pins_from = leftout_pins[leftout_pins.Pincode.isin(missing_dist_from)]
leftout_pins_from = leftout_pins_from[~leftout_pins_from.duplicated(subset=['Pincode', 'District', 'State'])]

In [40]:
leftout_pins_to = leftout_pins[leftout_pins.Pincode.isin(missing_dist_to)]

In [41]:
leftout_pins_to = leftout_pins_to[~leftout_pins_to.duplicated(subset=['Pincode', 'District', 'State'])]

In [103]:
#mapping values for left out pin - origin:
for i in leftout_pins_from['Pincode']:
    eway.From_District = np.where(eway.From_Pin == i,leftout_pins_from[leftout_pins_from['Pincode']==i].iloc[0,1],eway.From_District)
    eway.From_State = np.where(eway.From_Pin == i,leftout_pins_from[leftout_pins_from['Pincode']==i].iloc[0,2],eway.From_State)
    eway.From_Latitude = np.where(eway.From_Pin == i,leftout_pins_from[leftout_pins_from['Pincode']==i].iloc[0,3],eway.From_Latitude)
    eway.From_Longitude = np.where(eway.From_Pin == i,leftout_pins_from[leftout_pins_from['Pincode']==i].iloc[0,4],eway.From_Longitude)

In [42]:
#mapping values for left out pin - destination:
for i in leftout_pins_to['Pincode']:
    eway['To_District'] = np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,1],eway.To_District)
    eway['To_State'] = np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,2],eway.To_State)
    eway['To_Latitude'] = np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,3],eway.To_Latitude)
    eway['To_Longitude'] = np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,4],eway.To_Longitude)

In [51]:
eway.to_csv('eway_90_cleaned.csv',index=False)

In [43]:
eway.isna().sum()

date                  0
From_gst              0
From_company          0
From_Pin              0
From_District         0
From_State            0
From_Latitude         9
From_Longitude        9
To_GST                0
To_company          295
To_Pin                0
To_District           0
To_State              0
To_Latitude         284
To_Longitude        284
HSN Code              0
HSN_chapter           0
HSN_Chapter_Name      4
Assess Val.           0
veh_no                0
dtype: int64

In [134]:
leftout_pins_to.dtypes

Pincode        int64
District      object
State         object
Latitude     float64
Longitude    float64
dtype: object

In [135]:
np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,1],eway.To_District)

array(['SANGAREDDY', 'ANANTAPUR', 'SPSR NELLORE', ..., 'FARIDABAD',
       'JHAJJAR', 'FARIDABAD'], dtype=object)

In [138]:
leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,1]

'Chennai'

In [141]:
eway[eway.To_District.isna()].From_Pin.unique()

array([607105, 400071, 635109, 602106, 602117, 601201, 600001, 602105,
       631701, 635112, 642202, 506330, 600103, 632405, 603004, 641401,
       621316, 600124, 638312, 638007, 600128, 641658, 632602, 631604,
       636013, 600056, 626117, 641603, 641034, 626123, 600055, 600052,
       635126, 600066, 602004, 641107, 641017, 641607, 641031, 600119,
       638003, 641687, 641602, 600058, 641021, 600050, 600024, 600132,
       600067, 600123, 638008, 600044, 600069, 641030, 602003, 641062,
       638052, 641110, 600110, 600054, 603209, 604305, 602024, 603204,
       631402, 631561, 641653, 638102, 641604, 635751, 600095, 600098,
       625002, 635114, 632403, 602002, 600017, 600075, 641407, 600100,
       600003, 626189, 603105, 635802, 600048, 601301, 641005, 636006,
       637303, 603109, 641047, 625016, 631213, 638001, 641020, 641014,
       600078, 600099, 600073, 600002], dtype=int64)

In [145]:
leftout_pins_to[leftout_pins_to['Pincode']==607105].iloc[0,1]

'Kallakurichi'

In [6]:
eway=pd.read_csv('eway_90_cleaned.csv')
eway['date']= pd.to_datetime(eway['date'],errors='ignore')

In [23]:
pd.DataFrame(np.where(eway.To_Pin == i,leftout_pins_to[leftout_pins_to['Pincode']==i].iloc[0,1],eway.To_District)).isna().sum()

0    420
dtype: int64

In [35]:
leftout_pins_to.Pincode

0      600098
4      602004
30     607105
31     400071
32     635109
        ...  
127    641014
128    600078
129    600099
130    600073
131    600002
Name: Pincode, Length: 100, dtype: int64

In [38]:
missing_dist_to

array([517544, 515213, 831401, 524102, 530010, 524301, 517679, 517568,
       530035, 518504, 530023, 518503, 517546, 600027, 520014, 517509,
       520009, 518397, 600112, 517003, 523269, 517564, 520034, 495678,
       492007, 492006, 492021, 495679, 768232, 496552, 122106, 452004,
       320008, 325005, 500195, 134001, 122050, 122110, 123413, 123506,
       131201, 125123, 122022, 602015, 134002, 123503, 123505],
      dtype=int64)

In [47]:
eway[eway.To_Longitude.isna()].To_District.unique()

array(['SANGAREDDY', 'KURNOOL', 'EAST GODAVARI', 'GUNTUR', 'KRISHNA',
       'KOLHAPUR', 'CHITTOOR', 'MEDAK', 'VISAKHAPATANAM', 'HYDERABAD',
       'WANAPARTHY', 'KANKER', 'DANTEWADA', 'PUNE', 'CHURACHANDPUR',
       'MEDCHAL MALKAJGIRI', 'SHAHJAHANPUR', 'PALWAL', 'PANIPAT',
       'KURUKSHETRA'], dtype=object)

In [48]:
leftout_locations = {
    'District': ['SANGAREDDY', 'KURNOOL', 'EAST GODAVARI', 'GUNTUR', 'KRISHNA',
                 'KOLHAPUR', 'CHITTOOR', 'MEDAK', 'VISAKHAPATANAM', 'HYDERABAD',
                 'WANAPARTHY', 'KANKER', 'DANTEWADA', 'PUNE', 'CHURACHANDPUR',
                 'MEDCHAL MALKAJGIRI', 'SHAHJAHANPUR', 'PALWAL', 'PANIPAT',
                 'KURUKSHETRA'],
    'Latitude': [17.6245, 15.8281, 16.8393, 16.3065, 16.5062, 16.7041, 13.2177, 18.0453, 17.6868, 17.3850,
                 16.3644, 20.2710, 18.2491, 18.5204, 24.3333, 17.5064, 27.8804, 28.1470, 29.3909, 29.9696],
    'Longitude': [78.0866, 78.0342, 82.2967, 80.4365, 80.6480, 74.2433, 79.1013, 78.2636, 83.2185, 78.4867,
                  78.0648, 81.6675, 81.0965, 73.8567, 93.6713, 78.5419, 79.9182, 77.3299, 76.9635, 76.8753]
}

In [49]:
leftout_locationsdf=pd.DataFrame(leftout_locations)

In [50]:
leftout_locationsdf

,District,Latitude,Longitude
0,SANGAREDDY,17.6245,78.0866
1,KURNOOL,15.8281,78.0342
2,EAST GODAVARI,16.8393,82.2967
3,GUNTUR,16.3065,80.4365
4,KRISHNA,16.5062,80.6480
5,KOLHAPUR,16.7041,74.2433
6,CHITTOOR,13.2177,79.1013
7,MEDAK,18.0453,78.2636
8,VISAKHAPATANAM,17.6868,83.2185
9,HYDERABAD,17.3850,78.4867


In [52]:
for i in leftout_locationsdf['District']:
    eway['To_Latitude'] = np.where(eway.To_District == i,leftout_locationsdf[leftout_locationsdf['District']==i].iloc[0,1],eway.To_Latitude)
    eway['To_Longitude'] = np.where(eway.To_District == i,leftout_locationsdf[leftout_locationsdf['District']==i].iloc[0,2],eway.To_Longitude)

In [53]:
eway.isna().sum()

date                  0
From_gst              0
From_company          0
From_Pin              0
From_District         0
From_State            0
From_Latitude         9
From_Longitude        9
To_GST                0
To_company          295
To_Pin                0
To_District           0
To_State              0
To_Latitude           0
To_Longitude          0
HSN Code              0
HSN_chapter           0
HSN_Chapter_Name      4
Assess Val.           0
veh_no                0
dtype: int64

In [109]:
eway.to_csv('eway_cleaned1.csv', index = False)

In [79]:
iqr = np.quantile(eway['Assess Val.'],0.75) - np.quantile(eway['Assess Val.'],0.25)

In [81]:
eway[eway['Assess Val.']>iqr]

,date,From_gst,From_company,From_Pin,From_District,From_State,From_Latitude,From_Longitude,To_GST,To_company,To_Pin,To_District,To_State,To_Latitude,To_Longitude,HSN Code,HSN_chapter,HSN_Chapter_Name,Assess Val.,veh_no
7,2021-08-01,33AAACH2364M1ZM,HYUNDAI MOTOR INDIA LIMITED,602117,KANCHIPURAM,TAMIL NADU,12.925139,80.198694,37AAECS8044H1Z5,SRI JAYALAKSHMI AUTOMOTIVES PVT LTD,530003,VISAKHAPATANAM,ANDHRA PRADESH,17.686800,83.218500,87032291,87,Tractors & Motor Vehicles,588530.00,HR55Q7837
10,2021-08-01,33AAACH2364M1ZM,HYUNDAI MOTOR INDIA LIMITED,602117,KANCHIPURAM,TAMIL NADU,12.925139,80.198694,37AAECS8044H1Z5,SRI JAYALAKSHMI AUTOMOTIVES PVT LTD,530007,VISAKHAPATANAM,ANDHRA PRADESH,17.686800,83.218500,87032291,87,Tractors & Motor Vehicles,570990.00,HR55AD4960
12,2021-08-02,33AAACV3007D1Z1,VISHNUPRIA PAPER MILL PRIVATE LIMITED,600067,THIRUVALLUR,TAMIL NADU,13.530000,80.125000,37AAJFG3935H1Z8,GIRNAR PACKAGING,524101,SPSR NELLORE,ANDHRA PRADESH,14.146361,79.846944,23022090,23,"Flours, Meals & Pellets",590745.00,TN21AZ8062
15,2021-08-01,33AAACA4651L1ZT,ASHOK LEYLAND LIMITED,635109,KRISHNAGIRI,TAMIL NADU,12.747362,77.842436,37AAACA3428K1ZQ,AUTOMOTIVE MANUFACTURERS PVTLTD,521104,KRISHNA,ANDHRA PRADESH,16.506200,80.648000,87060042,87,Tractors & Motor Vehicles,2753906.00,TN70TC0229
16,2021-08-01,33AAACA4651L1ZT,ASHOK LEYLAND LIMITED,635109,KRISHNAGIRI,TAMIL NADU,12.747362,77.842436,37AAACA3428K1ZQ,AUTOMOTIVE MANUFACTURERS PVTLTD,521104,KRISHNA,ANDHRA PRADESH,16.506200,80.648000,87060042,87,Tractors & Motor Vehicles,2753906.00,TN70TC0216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16904,2021-08-02,33AABCB7140C1ZC,BMW INDIA PRIVATE LIMITED,603002,CHENGALPATTU,TAMIL NADU,12.701279,80.010161,06AAGCD4246E1ZX,DEUTSCHE CARS PRIVATE LIMITED,121003,FARIDABAD,HARYANA,77.264444,28.462500,870332,87,Tractors & Motor Vehicles,4290696.23,NL01K8004
16905,2021-08-02,33AABCB7140C1ZC,BMW INDIA PRIVATE LIMITED,603002,CHENGALPATTU,TAMIL NADU,12.701279,80.010161,06AAGCD4246E1ZX,DEUTSCHE CARS PRIVATE LIMITED,121003,FARIDABAD,HARYANA,77.264444,28.462500,870323,87,Tractors & Motor Vehicles,6968489.23,NL01K8004
16906,2021-08-02,33AABCB7140C1ZC,BMW INDIA PRIVATE LIMITED,603002,CHENGALPATTU,TAMIL NADU,12.701279,80.010161,06AAGCD4246E1ZX,DEUTSCHE CARS PRIVATE LIMITED,121003,FARIDABAD,HARYANA,77.264444,28.462500,870323,87,Tractors & Motor Vehicles,5480134.52,NL01K8004
16907,2021-08-02,33AABCB7140C1ZC,BMW INDIA PRIVATE LIMITED,603002,CHENGALPATTU,TAMIL NADU,12.701279,80.010161,06AAGCD4246E1ZX,DEUTSCHE CARS PRIVATE LIMITED,121003,FARIDABAD,HARYANA,77.264444,28.462500,870323,87,Tractors & Motor Vehicles,3356582.39,NL01K8004


In [85]:
eway['Assess Val.'].describe()

count    1.691100e+04
mean     6.042629e+05
std      2.651228e+06
min      5.000000e+04
25%      9.449288e+04
50%      1.941318e+05
75%      5.469210e+05
max      1.782005e+08
Name: Assess Val., dtype: float64

In [92]:
eway[eway['Assess Val.']==eway['Assess Val.'].max()]

,date,From_gst,From_company,From_Pin,From_District,From_State,From_Latitude,From_Longitude,To_GST,To_company,To_Pin,To_District,To_State,To_Latitude,To_Longitude,HSN Code,HSN_chapter,HSN_Chapter_Name,Assess Val.,veh_no
15257,2021-08-03,33AAACX1645B1ZV,AAACX1645B,600124,THIRUVALLUR,TAMIL NADU,13.076,79.979,06AAACX1645B1ZS,XIAOMI TECHNOLOGY INDIA PVT LTD,124108,JHAJJAR,HARYANA,28.735904,76.510506,85171211,85,Electrical Parts & Electronics,1.782005e+08,KA01AG9047


In [2]:
eway=pd.read_csv('eway_cleaned.csv')

In [7]:
eway['From_District']=eway.From_District.str.upper()

## <span style = 'color:#0b5345'> Research Questions</span>

In [ ]:
#general questions#

#how many transportations take place on an average in one single day
#how many transportations take place from each district on an average in one single day
#top 10 transported commodities
#top 10 transported companies
#which end district has more demand  -- to develop the goods shed present there
#which starting district has more demand  -- to develop the goods shed present in the starting point

#top 5 commodities out of each district

#for infra development#

# from district wise average value of goods transported sorted in descending
# to district wise "
# What commodities are being transported from a district - how many transportations of the same
# What commodities are being transported to a district - how many transportations of the same

#for approaching companies
#top 5 companies based on value of transportation in each districts
#transportations of motorvehicle and trucks (considering this as a target commoditiy)

#one company transports to which companies and districts --- #better to see it in visual as there are lots of companies

#### <span style = 'color:#0b5345'>How many transportations take place on an average in one single day? </span> 

In [18]:
eway.groupby('date')['veh_no'].count()

date
2021-08-01    1275
2021-08-02    5872
2021-08-03    6239
2021-08-04    3525
Name: veh_no, dtype: int64

<b> On an average, around 5000 trasnportations take palce every day from TamilNadu.</b>

(Ignoring 01-08-2023, as the dataset majorly consist of transportations took place in 3 days)

#### <span style = 'color:#0b5345'>How many transportations take place from each district on an average in one single day? </span> 

In [10]:
(eway.groupby(['From_District'])['veh_no'].count()/eway.date.nunique()).sort_values(ascending=False).head(30)

From_District
THIRUVALLUR        1136.00
CHENNAI             666.00
KANCHIPURAM         572.25
TIRUPPUR            316.50
KRISHNAGIRI         248.00
COIMBATORE          221.25
CHENGALPATTU        216.75
ERODE               183.50
SALEM               147.00
VIRUDHUNAGAR         78.50
MADURAI              54.75
RANIPET              53.00
TIRUCHIRAPPALLI      40.00
NAMAKKAL             39.25
TUTICORIN            27.75
VELLORE              25.25
THENI                21.00
DINDIGUL             20.50
TIRUPATHUR           20.00
CUDDALORE            17.25
KARUR                13.75
KANNIYAKUMARI         9.50
TIRUNELVELI           8.75
TIRUVANNAMALAI        8.75
VILLUPURAM            8.50
PERAMBALUR            7.50
ARIYALUR              6.50
ANANTAPUR             6.50
PONDICHERRY           5.25
SPSR NELLORE          4.75
Name: veh_no, dtype: float64

<b>A majority of trasportations take place from the Tiruvallur district which is around 2 times higher than the chennai district that comes next </b>

#### <span style = 'color:#0b5345'>List the top 10 transported commodities by number of transportations </span> 

In [11]:
eway.HSN_Chapter_Name.value_counts().sort_values(ascending=False).head(10)

Tractors & Motor Vehicles                          3167
Industrial Machinery                               2111
Electrical Parts & Electronics                     2053
Men & Women Clothing                               1005
Cotton materials, Synthetics & Woven fabrics        820
Polymers, Polyethylene, Cellulose                   531
Iron tube, piles & Sheets                           524
Medical, Chemical & Astronomy                       490
Rubber, Plates, Belt, Condesnsed Milk               462
Synthetic felament tows & Polyster staple fiber     432
Name: HSN_Chapter_Name, dtype: int64

<b>Tractors & Motor vehicles, Industrial machinery and Electrical parts & electronics form the major part of trasportations taken place. All these consignments can be transported in bulk through railways than Road.</b>

The results might also have an impact from the fact that higher number of small consignments can be transported in a single trip through a single v 

#### <span style = 'color:#0b5345'>List the top 10 transported commodities by value of transportations </span> 

In [13]:
eway.groupby('HSN_Chapter_Name')['Assess Val.'].sum().sort_values(ascending=False).head(10)

HSN_Chapter_Name
Tractors & Motor Vehicles                       2.227415e+09
Electrical Parts & Electronics                  2.149999e+09
Industrial Machinery                            1.395632e+09
Pearls, Diamonds, Gold, Platinum                3.651568e+08
Medical, Chemical & Astronomy                   3.424054e+08
Iron, Alloys, Scrap & Granules                  3.264955e+08
Polymers, Polyethylene, Cellulose               3.129545e+08
Rubber, Plates, Belt, Condesnsed Milk           2.852622e+08
Cotton materials, Synthetics & Woven fabrics    2.573397e+08
Iron tube, piles & Sheets                       1.697924e+08
Name: Assess Val., dtype: float64

<b>Tractors & Motor vehicles, Industrial machinery and Electrical parts & electronics form the major part of trasportations taken place here as well. Nonetheless other consignments present in the list can also be moved through Railways. Pearls, Diamonds, Gold, Platinum might not be potential commodity for approaching as it requires a lot of security and are highly unlikely by companies to transport it through rail.

#### <span style = 'color:#0b5345'>List the top 10 transported companies by the number of transportations </span> 

In [15]:
#top 10 transported companies  - number of transportations
eway.From_company.value_counts().sort_values(ascending=False).head(10)

HYUNDAI MOTOR INDIA LIMITED                             1098
ASHOK LEYLAND LIMITED                                    323
SUNDRAM FASTENERS LIMITED                                308
APPARIO RETAIL PRIVATE LIMITED                           290
RENAULT INDIA PRIVATE LIMITED                            263
DARSHITA AASHIYANA PRIVATE LIMITED                       206
RISING STARS MOBILE INDIA PRIVATE LIMITED                143
DELL INTERNATIONAL SERVICES INDIA PRIVATE LIMITEDM-4     124
SAVEX TECHNOLOGIES PRIVATE LIMITED                       123
MRF LIMITED                                              123
Name: From_company, dtype: int64

<b>Hyundai motors have performed the highest number of trasportations followed by Ashok leyland limited and sundaram fastners limited. These companies transport majorly manufacture vehicles and mechanical parts</b>

#### <span style = 'color:#0b5345'>List the top 10 transported companies by the value of transportations </span> 

In [17]:
eway.groupby('From_company')['Assess Val.'].sum().sort_values(ascending=False).head(10)

From_company
HYUNDAI MOTOR INDIA LIMITED                         8.817060e+08
ASHOK LEYLAND LIMITED                               5.887959e+08
RISING STARS MOBILE INDIA PRIVATE LIMITED           5.706779e+08
LALITHAA JEWELLERY MART PRIVATE LIMITED             3.510855e+08
APPLE INDIA PRIVATE LIMITED                         2.491195e+08
ORISSA STEVEDORES LIMITED                           2.146461e+08
FLEXTRONICS TECHNOLOGIES (INDIA) PRIVATE LIMITED    2.004383e+08
AAACX1645B                                          1.914902e+08
ITC LIMITED                                         1.173160e+08
RENAULT INDIA PRIVATE LIMITED                       1.144661e+08
Name: Assess Val., dtype: float64

<b>Highest value of transportation has been made by HYUNDAI MOTOR INDIA LIMITED, followed by ASHOK LEYLAND LIMITED, most of the top transportations consist of companies which manufacture vehicles, mechanical and electrical components </b>

#### <span style = 'color:#0b5345'>'From' District wise average value of goods transported per day</span> 

In [47]:
# from district wise average value of goods transported per day
(eway.groupby('From_District')['Assess Val.'].sum()/eway.date.nunique()).sort_values(ascending=False)

From_District
THIRUVALLUR            6.691693e+08
KANCHIPURAM            5.600105e+08
CHENNAI                3.788051e+08
KRISHNAGIRI            2.281018e+08
CHENGALPATTU           1.464522e+08
                           ...     
RAJKOT                 4.693750e+04
ALWAR                  4.312875e+04
GAUTAM BUDDHA NAGAR    3.519840e+04
FARIDABAD              2.750000e+04
GUNTUR                 1.265625e+04
Name: Assess Val., Length: 82, dtype: float64

<b>On an average, highest value of goods is being transported from Tiruvallur district, followed by Kanchipuram and Chennai</b>

#### <span style = 'color:#0b5345'>'To' District wise average value of goods transported per day</span> 

In [49]:
# To district wise average value of goods transported per day
(eway.groupby('To_District')['Assess Val.'].sum()/eway.date.nunique()).sort_values(ascending=False)

To_District
CHITTOOR        3.044082e+08
KRISHNA         2.672410e+08
GURUGRAM        2.290193e+08
ANANTAPUR       1.731584e+08
JHAJJAR         1.333330e+08
                    ...     
NAGAPATTINAM    1.300000e+04
NAGAUR          1.296000e+04
DHUBRI          1.257120e+04
SHAHJAHANPUR    1.255150e+04
BHAGALPUR       1.255150e+04
Name: Assess Val., Length: 211, dtype: float64

<b>On an average, highest value of goods is being transported to Chitoor district, followed by Krishna and Gurugram</b>

#### <span style = 'color:#0b5345'>commodities transported more than 50 times in 4 days in the top 10 highly transporting districts</span> 

In [119]:
top_10_dist = eway.From_District.value_counts().sort_values(ascending = False).head(10).index
comm_wise =pd.DataFrame(eway.groupby(['From_District','HSN_Chapter_Name'])['HSN_Chapter_Name'].count()).rename(columns={'HSN_Chapter_Name':'count'}).reset_index()
comm_wise[(comm_wise.From_District.isin(top_10_dist)) & (comm_wise['count']>50)].sort_values(['From_District','count'],ascending=[True,False]).style.hide_index()

C:\Users\Sriram\AppData\Local\Temp\ipykernel_1472\2498822136.py:4: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  comm_wise[(comm_wise.From_District.isin(top_10_dist)) & (comm_wise['count']>50)].sort_values(['From_District','count'],ascending=[True,False]).style.hide_index()


From_District,HSN_Chapter_Name,count
CHENGALPATTU,Tractors & Motor Vehicles,239
CHENGALPATTU,Industrial Machinery,121
CHENGALPATTU,Electrical Parts & Electronics,98
CHENNAI,Industrial Machinery,362
CHENNAI,Electrical Parts & Electronics,358
CHENNAI,"Iron tube, piles & Sheets",167
CHENNAI,"Polymers, Polyethylene, Cellulose",157
CHENNAI,"Medical, Chemical & Astronomy",155
CHENNAI,Drugs & Pharmaceuticals,145
CHENNAI,"Insecticides, Artificial Carbon & Graphite",117


#### <span style = 'color:#0b5345'>commodities transported more than 50 times in 4 days to top 10 districts(destinations)</span>

In [121]:
#commodities transported more than 50 times in 4 days to top 10 districts(destinations)
top_10_to_dist = eway.To_District.value_counts().sort_values(ascending = False).head(10).index
comm_wise =pd.DataFrame(eway.groupby(['To_District','HSN_Chapter_Name'])['HSN_Chapter_Name'].count()).rename(columns={'HSN_Chapter_Name':'count'}).reset_index()
comm_wise[(comm_wise.To_District.isin(top_10_to_dist)) & (comm_wise['count']>50)].sort_values(['To_District','count'],ascending=[True,False]).style.hide_index()

C:\Users\Sriram\AppData\Local\Temp\ipykernel_1472\1060703190.py:4: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  comm_wise[(comm_wise.To_District.isin(top_10_to_dist)) & (comm_wise['count']>50)].sort_values(['To_District','count'],ascending=[True,False]).style.hide_index()


To_District,HSN_Chapter_Name,count
ANANTAPUR,Tractors & Motor Vehicles,460
ANANTAPUR,Industrial Machinery,145
ANANTAPUR,Electrical Parts & Electronics,89
ANANTAPUR,"Iron, Alloys, Scrap & Granules",51
CENTRAL,"Cotton materials, Synthetics & Woven fabrics",126
CENTRAL,"Medical, Chemical & Astronomy",62
CENTRAL,Synthetic felament tows & Polyster staple fiber,61
CENTRAL,Industrial Machinery,57
CHITTOOR,Electrical Parts & Electronics,236
CHITTOOR,Industrial Machinery,198


#### <span style = 'color:#0b5345'>top 5 companies based on value of transportation in each district of chennai division of railways</span>

In [94]:
eway_ranks = eway.copy()
eway_ranks['sum']=eway.groupby(['From_District','From_company'])['Assess Val.'].transform('sum')
eway_ranks['ranks']=eway_ranks.groupby('From_District')['sum'].rank(method='dense',ascending=False)


In [101]:
top5 = eway_ranks[eway_ranks.ranks<=5]
top5_alldist = top5[['From_District','From_company','sum','ranks']].drop_duplicates().sort_values(by=['From_District','sum'],ascending=[True,False])
top5_alldist[top5_alldist.From_District.isin(["CHENNAI", "RANIPET", "KRISHNAGIRI", "KANCHIPURAM", "CHENGALPATTU", "TIRUVANNAMALAI", "VELLORE"])].reset_index(drop=True)
#"CHENNAI", "RANIPET", "KRISHNAGIRI", "KANCHIPURAM", "CHENGALPATTU", "TIRUVANNAMALAI", "VELLORE"

,From_District,From_company,sum,ranks
0,CHENGALPATTU,BMW INDIA PRIVATE LIMITED,9.585942e+07,1.0
1,CHENGALPATTU,FORD INDIA PRIVATE LIMITED,9.172179e+07,2.0
2,CHENGALPATTU,DATA PATTERNS INDIA PRIVATE LIMITED,3.113500e+07,3.0
3,CHENGALPATTU,SILICON LABS PRIVATE LIMITED,2.676151e+07,4.0
4,CHENGALPATTU,BRAKES INDIA PRIVATE LIMITED,2.405090e+07,5.0
5,CHENNAI,LALITHAA JEWELLERY MART PRIVATE LIMITED,3.510855e+08,1.0
6,CHENNAI,ORISSA STEVEDORES LIMITED,2.146461e+08,2.0
7,CHENNAI,INDIAN OIL CORPORATION LIMITED,3.947111e+07,3.0
8,CHENNAI,HCL TECHNOLOGIES LIMITED,2.065292e+07,4.0
9,CHENNAI,SATTVA HI-TECH AND CONWARE PRIVATE LIMITED,1.834180e+07,5.0


#### <span style = 'color:#0b5345'>companies that transport motorvehicle and trucks (considering this as a target commoditiy)</span>

In [ ]:
mv = eway[eway.HSN_Chapter_Name=='Tractors & Motor Vehicles']
mv.groupby('From_company')['Assess Val.'].sum().sort_values(ascending=False).head(15)

### <span style = 'color:#0b5345'>Conclusion</span>

The actionable insights derived from this analysis provide a solid foundation for informed decision-making, resource allocation, and policy development to enhance efficiency, reduce bottlenecks, and foster a more agile and responsive transportation network.  Further visualization of the data is carried out in Power BI to delve deeper in to the data and to see trends taking place even in the minute level.